In [1]:
import importlib
import torch

from dataLoader.DataLoader import get_data_loader
from utils.Constants import tokenizer

from models.QABert import QABertTrainer
from train.Trainer import train_epoch

In [2]:
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'


Failed to find GPU. Will use CPU.


In [19]:
import dataLoader.DataLoader
import dataLoader.DataLoaderUtils


importlib.reload(dataLoader.DataLoader)
importlib.reload(dataLoader.DataLoaderUtils)

from dataLoader.DataLoader import get_data_loader
from dataLoader.DataLoaderUtils import get_question_answers_for_where_value_def_length, get_question_answers_def_length

dev_data_loader = get_data_loader(data_type = 'dev', tokenizer = tokenizer, batch_size = 1)

100%|██████████| 8421/8421 [00:40<00:00, 207.94it/s]


We have 8421 dev data with 2716 tables.


In [26]:
import utils.Constants
import models.WhereRanker

importlib.reload(models.WhereRanker)
importlib.reload(utils.Constants)

from models.WhereRanker import WhereRankerTrainer

where_ranker = WhereRankerTrainer(device, dev_data_loader)

iterator = iter(dev_data_loader)
for i in range(5):
    d = next(iterator)

input_ids = d["input_ids"].to(device)
attention_mask = d["attention_mask"].to(device)
token_type_ids = d["token_type_ids"].to(device)

agg_target = d["target"]['WHERE'].to(device)

agg_output = where_ranker.predict(
    input_ids,
    attention_mask,
    token_type_ids
)

where_ranker.calc_loss(
    agg_output, agg_target
)

In [ ]:
import models.SelectRanker
import models.WhereRanker
import train.Trainer
import models.AggregationClassifier
import models.WhereConditionClassifier
import models.WhereNumberClassifier

importlib.reload(models.QABert)
importlib.reload(models.SelectRanker)
importlib.reload(models.WhereRanker)
importlib.reload(models.WhereConditionClassifier)
importlib.reload(models.WhereNumberClassifier)
importlib.reload(models.AggregationClassifier)
importlib.reload(train.Trainer)

from models.QABert import QABertTrainer
from models.SelectRanker import SelectRankerTrainer
from models.WhereRanker import WhereRankerTrainer
from models.WhereConditionClassifier import WhereConditionClassifierTrainer
from models.WhereNumberClassifier import WhereNumberClassifierTrainer
from models.AggregationClassifier import AggregationClassifierTrainer
from train.Trainer import train_epoch

models = dict(
    selection_trainer = SelectRankerTrainer(device, dev_data_loader),
    agg_class_trainer = AggregationClassifierTrainer(device, dev_data_loader),
    where_ranker_trainer = WhereRankerTrainer(device, dev_data_loader),
    where_cond_class_trainer = WhereConditionClassifierTrainer(device, dev_data_loader),
    where_numb_class_trainer = WhereNumberClassifierTrainer(device, dev_data_loader),
    qa_trainer = QABertTrainer(device, dev_data_loader),
)

train_epoch(models, dev_data_loader, device)